# Base functions

In [1]:
import numpy as np 
from evalkit.basic import Indvdl, Population, TaskType

#create description
description = {"Name1":{"quantity":[1], "values":[[0,1]]},
               "Name2":{"quantity":[1,2,3,4,5], "values":[[0, 1],[0, 1]]}
               }
#fitness function
def allone(indvdl):
    count_values = 0
    sum_values = 0
    for key in indvdl.keys():
        count_values += np.array(indvdl[key]).size
        sum_values += np.sum(indvdl[key])
    return sum_values/count_values

In [2]:
from evalkit.geal import inits

#create conteiner of population
pop = Population(description=description, evaluate=allone, fitness_target = 0.9)
#create a population of 10 individuals.
pop += inits.initRandom(pop.description,size=10)
# evaluate indidivuals
pop.run_evaluate()
pop

[{'Name1': [[1]], 'Name2': [[0, 1], [0, 0]]},
 {'Name1': [[0]], 'Name2': [[0, 0], [1, 0], [1, 1], [1, 1]]},
 {'Name1': [[0]], 'Name2': [[0, 0], [0, 0], [1, 0]]},
 {'Name1': [[1]], 'Name2': [[1, 1]]},
 {'Name1': [[1]], 'Name2': [[0, 0], [1, 1]]},
 {'Name1': [[1]], 'Name2': [[1, 0]]},
 {'Name1': [[0]], 'Name2': [[0, 0], [0, 1], [1, 0]]},
 {'Name1': [[1]], 'Name2': [[1, 1], [1, 0], [1, 1], [0, 1], [0, 0]]},
 {'Name1': [[0]], 'Name2': [[0, 1], [1, 1], [0, 1]]},
 {'Name1': [[1]], 'Name2': [[0, 1], [1, 0], [0, 0], [0, 1], [1, 1]]}]

In [3]:
from evalkit.geal import sels

#run tournament selection
indvdls = np.random.choice(pop, size=3)
sel_indvdls = sels.selTournament(indvdls, pop.fitness_target, pop.task_type )
pop += sel_indvdls
sel_indvdls

[{'Name1': [[0]], 'Name2': [[0, 1], [1, 1], [0, 1]]}]

In [4]:
from evalkit.geal import cross

#crossover individual 0 and individual 1
print(f"individual0 = {pop[0]}")
print(f"individual1 = {pop[1]}")
pop[0], pop[1] = cross.crosOnePoint(pop[0],pop[1])
print(f"individual0 = {pop[0]}")
print(f"individual1 = {pop[1]}")

individual0 = {'Name1': [[1]], 'Name2': [[0, 1], [0, 0]]}
individual1 = {'Name1': [[0]], 'Name2': [[0, 0], [1, 0], [1, 1], [1, 1]]}
individual0 = {'Name1': [[0]], 'Name2': [[0, 0], [0, 0]]}
individual1 = {'Name1': [[1]], 'Name2': [[0, 1], [1, 0], [1, 1], [1, 1]]}


In [5]:
from evalkit.geal import muts

#mutate individual 0
print("individual = ", pop[0])
mut_indvdl = muts.mutRand(pop[3], pop.description, sel_keys="rand")
print("mut_indvdl = ", mut_indvdl)

individual =  {'Name1': [[0]], 'Name2': [[0, 0], [0, 0]]}
mut_indvdl =  [{'Name1': [[0]], 'Name2': [[0, 0], [0, 1]]}]


# Algorithms

In [6]:
from evalkit.geal import algs
from evalkit.tools.statslog import StatsLog
import numpy as np
import time
import plotly.express as pe




# Create statslog functions
def pop_len(pop):
    return len(pop)

def avg_time(pop):
    array_times = [ pop[indvdl_indx].evaltime for indvdl_indx in range(len(pop)) ]
    return np.sum(array_times)/len(array_times)

def std_fit(pop):
    fitnesses = [ pop[indvdl_indx].fitness for indvdl_indx in range(len(pop)) ]
    return np.std(fitnesses)

def min_fit(pop):
    fitnesses = [ pop[indvdl_indx].fitness for indvdl_indx in range(len(pop)) ]
    return np.min(fitnesses)

def max_fit(pop):
    fitnesses = [ pop[indvdl_indx].fitness for indvdl_indx in range(len(pop)) ]
    return np.max(fitnesses)

def best_indvdls(pop):
    return pop.best_indvdls()

# Min algorithm

In [7]:
def condition1(indvdl):
    pass

def condition2(indvdl):
    pass

description = {"Name1":{"quantity":[1], "values":[[0,1]], "conditions":[[]]},
               "Name2":{"quantity":range(1,100), "values":[[0, 1],[0, 1], [0,1], [0,1]], "condition":[[]]}
               }

pop_size=20
pop = Population(description=description, evaluate=allone, fitness_target = 0, task_type=TaskType.MIN)
pop.extend( inits.initRandom(pop.description,size=pop_size) )

#create statslog
sl = StatsLog(funcs=[pop_len, avg_time, std_fit, min_fit, max_fit, best_indvdls], viznt_funcs=[best_indvdls])

#start algorithm randwhilefalse
sel_size = int(np.round(pop_size*0.4))
cros_size = int(np.round(pop_size*0.3)) # couples
mut_size = int(np.round(pop_size*0.3))
best_pop = algs.randwhilefalse(pop, sel_size=sel_size, tour_size=2, cros_size=cros_size, mut_size=mut_size, statslog=sl)
print(f"avg_time = ", np.sum([ sl.log[log_key]["avg_time"] for log_key in sl.log ]) )

AttributeError: 'StatsLog' object has no attribute '_StatsLog__doc'

In [8]:
best_pop.best_indvdls()

[{'Name1': [[0]], 'Name2': [[0, 0, 0, 0], [0, 0, 0, 0]]}]

In [9]:
sl.log[1]

{'pop_len': 20,
 'avg_time': 0.00024279356002807618,
 'std_fit': 0.08178437875953726,
 'min_fit': 0.37735849056603776,
 'max_fit': 0.8,
 'best_indvdls': [{'Name1': [[0]],
   'Name2': [[1, 0, 0, 1],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [0, 0, 0, 0],
    [1, 0, 0, 1],
    [1, 0, 0, 1],
    [0, 1, 0, 0],
    [0, 1, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [1, 0, 0, 0],
    [1, 1, 1, 0],
    [1, 0, 0, 0]]}]}

In [10]:
fig = pe.line(x=list(sl.log.keys()), y=[ sl.log[key]["min_fit"] for key in sl.log])
fig.show()

In [11]:
description = {"Name1":{"quantity":[1], "values":[[0,1]]},
               "Name2":{"quantity":range(1,100), "values":[[0, 1],[0, 1], [0,1], [0,1]]}
               }

pop_size=20
pop = Population(description=description, evaluate=allone, fitness_target = 0, task_type=TaskType.MIN)
pop.extend( inits.initRandom(pop.description,size=pop_size) )

#create statslog
sl = StatsLog(funcs=[pop_len, avg_time, std_fit, min_fit, max_fit, best_indvdls], viznt_funcs=[best_indvdls])
    
#start algorithm randwhilefalse
sel_size = int(np.round(pop_size*0.4))
cros_size = int(np.round(pop_size*0.3)) # couples
mut_size = int(np.round(pop_size*0.3))
best_pop = algs.randngen(pop, ngen=10, sel_size=sel_size, tour_size=2, cros_size=cros_size, mut_size=mut_size, statslog=sl)
print(f"avg_time = ", np.sum([ sl.log[log_key]["avg_time"] for log_key in sl.log ]) )

gen|  pop_len|  avg_time|  std_fit|  min_fit|  max_fit|  
1|  20|  0.00023430585861206055|  0.0664661901670617|  0.35294117647058826|  0.6588235294117647|  
2|  20|  0.00015954971313476564|  0.07539068881607075|  0.2|  0.5590062111801242|  
3|  20|  0.0002209782600402832|  0.03297307600825749|  0.4|  0.5280898876404494|  
4|  20|  0.00019692182540893556|  0.05953440458121311|  0.3076923076923077|  0.6097560975609756|  
5|  20|  0.00021734237670898438|  0.06260504231276164|  0.3076923076923077|  0.608|  
6|  20|  0.00020552873611450194|  0.05544857905406549|  0.3076923076923077|  0.5692307692307692|  
7|  20|  0.00018781423568725586|  0.03987136585001456|  0.3953488372093023|  0.5463414634146342|  
8|  20|  0.0002456545829772949|  0.043491165970932326|  0.3953488372093023|  0.5463917525773195|  
9|  20|  0.00019207000732421876|  0.07343330645926005|  0.2|  0.5555555555555556|  
avg_time =  0.0018601655960083009


In [12]:
best_pop.best_indvdls()

[{'Name1': [[0]], 'Name2': [[0, 0, 1, 0]]}]

In [13]:
sl.log[1]

{'pop_len': 20,
 'avg_time': 0.00023430585861206055,
 'std_fit': 0.0664661901670617,
 'min_fit': 0.35294117647058826,
 'max_fit': 0.6588235294117647,
 'best_indvdls': [{'Name1': [[1]],
   'Name2': [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 1]]}]}

In [14]:
fig = pe.line(x=list(sl.log.keys()), y=[ sl.log[key]["min_fit"] for key in sl.log])
fig.show()

# Max algorithm

In [15]:
description = {"Name1":{"quantity":[1], "values":[[0,1]]},
               "Name2":{"quantity":range(1,100), "values":[[0, 1],[0, 1], [0,1], [0,1]]}
               }

pop_size=20
pop = Population(description=description, evaluate=allone, fitness_target = 1, task_type=TaskType.MAX)
pop.extend( inits.initRandom(pop.description,size=pop_size) )

#create statslog
sl = StatsLog(funcs=[pop_len, avg_time, std_fit, min_fit, max_fit, best_indvdls], viznt_funcs=[best_indvdls])

#start algorithm randwhilefalse
sel_size = int(np.round(pop_size*0.4))
cros_size = int(np.round(pop_size*0.3)) # couples
mut_size = int(np.round(pop_size*0.3))
best_pop = algs.randwhilefalse(pop, sel_size=sel_size, tour_size=2, cros_size=cros_size, mut_size=mut_size, statslog=sl)
print(f"avg_time = ", np.sum([ sl.log[log_key]["avg_time"] for log_key in sl.log ]) )

gen|  pop_len|  avg_time|  std_fit|  min_fit|  max_fit|  
1|  20|  0.0002762436866760254|  0.0778592352559338|  0.24|  0.6444444444444445|  
2|  20|  0.0002868056297302246|  0.06199934678062115|  0.4|  0.6444444444444445|  
3|  20|  0.00020924806594848632|  0.06510373968394778|  0.4497354497354497|  0.6491228070175439|  
4|  20|  0.0005622744560241699|  0.06541166588386448|  0.4|  0.6444444444444445|  
5|  20|  0.0001417994499206543|  0.045672005754952893|  0.4426877470355731|  0.6444444444444445|  
6|  20|  0.00012242794036865234|  0.04860021806039619|  0.4117647058823529|  0.6097560975609756|  
7|  20|  0.00011192560195922851|  0.05500722066730117|  0.3877551020408163|  0.6097560975609756|  
8|  20|  0.00012161731719970704|  0.0668166965782003|  0.4146341463414634|  0.7192982456140351|  
9|  20|  0.00014666318893432617|  0.05657901732921669|  0.4666666666666667|  0.7192982456140351|  
10|  20|  0.00013527870178222656|  0.06458517567481709|  0.45614035087719296|  0.7192982456140351|  

In [16]:
best_pop.best_indvdls()

[{'Name1': [[1]], 'Name2': [[1, 1, 1, 1]]}]

In [17]:
sl.log[1]

{'pop_len': 20,
 'avg_time': 0.0002762436866760254,
 'std_fit': 0.0778592352559338,
 'min_fit': 0.24,
 'max_fit': 0.6444444444444445,
 'best_indvdls': [{'Name1': [[1]],
   'Name2': [[1, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 0, 1, 0],
    [1, 0, 1, 0],
    [1, 1, 1, 0],
    [1, 1, 0, 1],
    [1, 0, 1, 0],
    [1, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 1, 1, 1],
    [1, 0, 1, 0]]}]}

In [18]:
fig = pe.line(x=list(sl.log.keys()), y=[ sl.log[key]["max_fit"] for key in sl.log])
fig.show()

In [19]:
description = {"Name1":{"quantity":[1], "values":[[0,1]]},
               "Name2":{"quantity":range(1,100), "values":[[0, 1],[0, 1], [0,1], [0,1]]}
               }

pop_size=20
pop = Population(description=description, evaluate=allone, fitness_target = 1, task_type=TaskType.MAX)
pop.extend( inits.initRandom(pop.description,size=pop_size) )

#create statslog
sl = StatsLog(funcs=[pop_len, avg_time, std_fit, min_fit, max_fit, best_indvdls], viznt_funcs=[best_indvdls])

#start algorithm randwhilefalse
sel_size = int(np.round(pop_size*0.4))
cros_size = int(np.round(pop_size*0.3)) # couples
mut_size = int(np.round(pop_size*0.3))
best_pop = algs.randngen(pop, ngen=10, sel_size=sel_size, tour_size=2, cros_size=cros_size, mut_size=mut_size, statslog=sl)
print(f"avg_time = ", np.sum([ sl.log[log_key]["avg_time"] for log_key in sl.log ]) )

gen|  pop_len|  avg_time|  std_fit|  min_fit|  max_fit|  
1|  20|  0.0002694487571716309|  0.02683121972255691|  0.4461538461538462|  0.5412844036697247|  
2|  20|  0.0002774357795715332|  0.030074800319121492|  0.43609022556390975|  0.5785123966942148|  
3|  20|  0.00023766756057739258|  0.05392810329848876|  0.3333333333333333|  0.5785123966942148|  
4|  20|  0.00020610094070434571|  0.03474912717413122|  0.45864661654135336|  0.6124031007751938|  
5|  20|  0.00016629695892333984|  0.031568388800268514|  0.4679245283018868|  0.6114649681528662|  
6|  20|  0.00019636154174804688|  0.030850343374522513|  0.42857142857142855|  0.5785123966942148|  
7|  20|  0.0001710057258605957|  0.030359938213916162|  0.4339622641509434|  0.583756345177665|  
8|  20|  0.00016289949417114258|  0.03673952585999688|  0.4377358490566038|  0.6226415094339622|  
9|  20|  0.00017317533493041993|  0.02952629068714191|  0.4542124542124542|  0.5773195876288659|  
avg_time =  0.0018603920936584475


In [20]:
best_pop.best_indvdls()

[{'Name1': [[0]],
  'Name2': [[1, 1, 0, 1],
   [1, 1, 1, 0],
   [1, 0, 1, 1],
   [0, 0, 1, 1],
   [0, 1, 0, 1],
   [1, 0, 1, 0],
   [0, 1, 1, 1],
   [1, 0, 1, 1],
   [0, 0, 0, 1],
   [1, 1, 1, 0],
   [1, 1, 0, 1],
   [1, 0, 0, 1],
   [0, 0, 0, 0],
   [0, 0, 0, 1],
   [1, 1, 1, 0],
   [1, 0, 1, 0],
   [1, 0, 0, 1],
   [1, 1, 1, 1],
   [0, 0, 0, 0],
   [1, 1, 0, 1],
   [1, 1, 1, 0],
   [1, 1, 1, 0],
   [1, 1, 1, 0],
   [0, 1, 1, 0]]}]

In [21]:
sl.log[1]

{'pop_len': 20,
 'avg_time': 0.0002694487571716309,
 'std_fit': 0.02683121972255691,
 'min_fit': 0.4461538461538462,
 'max_fit': 0.5412844036697247,
 'best_indvdls': [{'Name1': [[1]],
   'Name2': [[0, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 0, 1, 0],
    [1, 0, 1, 1],
    [0, 0, 0, 1],
    [1, 1, 1, 0],
    [1, 1, 0, 1],
    [0, 1, 1, 1],
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 1, 1],
    [1, 0, 0, 1],
    [1, 0, 0, 1],
    [0, 1, 0, 1],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 1, 1, 1],
    [0, 1, 0, 1],
    [1, 1, 1, 0],
    [0, 0, 0, 1],
    [1, 1, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [1, 1, 1, 1],
    [0, 0, 1, 1],
    [1, 0, 1, 0],
    [1, 0, 1, 1],
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [1, 1, 0, 1],
    [0, 1, 1, 1],
    [1, 0, 1, 0],
    [0, 1, 1, 1],
    [0, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 1, 1],
    [0, 0, 0, 1],
    [1, 0, 1, 1

In [22]:
fig = pe.line(x=list(sl.log.keys()), y=[ sl.log[key]["max_fit"] for key in sl.log])
fig.show()

# Lim algorithm

In [23]:
description = {"Name1":{"quantity":[1], "values":[[0,1]]},
               "Name2":{"quantity":range(1,100), "values":[[0, 1],[0, 1], [0,1], [0,1]]}
               }

pop_size=20
pop = Population(description=description, evaluate=allone, fitness_target = 0.1, max_deviation=0.1,task_type=TaskType.LIM)
pop.extend( inits.initRandom(pop.description,size=pop_size) )

#create statslog
sl = StatsLog(funcs=[pop_len, avg_time, std_fit, min_fit, max_fit, best_indvdls], viznt_funcs=[best_indvdls])

#start algorithm randwhilefalse
sel_size = int(np.round(pop_size*0.4))
cros_size = int(np.round(pop_size*0.3)) # couples
mut_size = int(np.round(pop_size*0.3))
best_pop = algs.randwhilefalse(pop, sel_size=sel_size, tour_size=2, cros_size=cros_size, mut_size=mut_size, statslog=sl)
print(f"avg_time = ", np.sum([ sl.log[log_key]["avg_time"] for log_key in sl.log ]) )

gen|  pop_len|  avg_time|  std_fit|  min_fit|  max_fit|  
1|  20|  0.00015668869018554689|  0.03221305567486922|  0.45353159851301117|  0.5841584158415841|  
2|  20|  0.00015860795974731445|  0.026037903112754|  0.44981412639405205|  0.5615615615615616|  
3|  20|  0.00023179054260253907|  0.036201157861811065|  0.4423963133640553|  0.5688073394495413|  
4|  20|  0.0002382040023803711|  0.044703510667711006|  0.36|  0.594059405940594|  
5|  20|  0.0002064943313598633|  0.051578841070094715|  0.34146341463414637|  0.5481727574750831|  
6|  20|  0.00015645027160644532|  0.05061897659042471|  0.36|  0.5617977528089888|  
7|  20|  0.00019550323486328125|  0.04976114800096112|  0.36923076923076925|  0.5662650602409639|  
8|  20|  0.0001748800277709961|  0.057447125153846305|  0.37593984962406013|  0.6226415094339622|  
9|  20|  0.00012714862823486327|  0.05325289464039987|  0.36|  0.5538461538461539|  
10|  20|  0.000164794921875|  0.06401008924406171|  0.36|  0.6415094339622641|  
11|  20| 

In [24]:
print("best_indvdls = ", best_pop.best_indvdls())
print("fitness indvdl 0  = ", best_pop.best_indvdls()[0].fitness)

best_indvdls =  [{'Name1': [[0]], 'Name2': [[0, 0, 0, 1], [0, 0, 0, 0]]}]
fitness indvdl 0  =  0.1111111111111111


In [25]:
fig = pe.line(x=range(len(best_pop)), y=[ indvdl.fitness for indvdl in best_pop])
fig.show()

In [26]:
description = {"Name1":{"quantity":[1], "values":[[0,1]]},
               "Name2":{"quantity":range(1,100), "values":[[0, 1],[0, 1], [0,1], [0,1]]}
               }

pop_size=20
pop = Population(description=description, evaluate=allone, fitness_target = 0.1, max_deviation=0.1,task_type=TaskType.LIM)
pop.extend( inits.initRandom(pop.description,size=pop_size) )

#create statslog
sl = StatsLog(funcs=[pop_len, avg_time, std_fit, min_fit, max_fit, best_indvdls], viznt_funcs=[best_indvdls])

#start algorithm randwhilefalse
sel_size = int(np.round(pop_size*0.4))
cros_size = int(np.round(pop_size*0.3)) # couples
mut_size = int(np.round(pop_size*0.3))
best_pop = algs.randwhilefalse(pop, sel_size=sel_size, tour_size=2, cros_size=cros_size, mut_size=mut_size, statslog=sl)
print(f"avg_time = ", np.sum([ sl.log[log_key]["avg_time"] for log_key in sl.log ]) )

gen|  pop_len|  avg_time|  std_fit|  min_fit|  max_fit|  
1|  20|  0.00014410018920898437|  0.11361008830295272|  0.4472049689440994|  1.0|  
2|  20|  0.0002010345458984375|  0.03137616626114971|  0.445859872611465|  0.5739130434782609|  
3|  20|  0.0001515507698059082|  0.05519438310775471|  0.45454545454545453|  0.7058823529411765|  
4|  20|  0.0001572251319885254|  0.042772959791194644|  0.4563758389261745|  0.6470588235294118|  
5|  20|  0.00023595094680786132|  0.039118361132120756|  0.4508670520231214|  0.6238532110091743|  
6|  20|  0.00015659332275390624|  0.037407570406850964|  0.4117647058823529|  0.5779816513761468|  
7|  20|  0.000126039981842041|  0.07932115186043324|  0.4444444444444444|  0.8235294117647058|  
8|  20|  0.00013660192489624025|  0.058113454113484876|  0.29411764705882354|  0.6153846153846154|  
9|  20|  0.00015627145767211915|  0.03959134026405929|  0.42696629213483145|  0.6|  
10|  20|  0.00011292695999145507|  0.040324055717543494|  0.35294117647058826|  

In [27]:
print("best_indvdls = ", best_pop.best_indvdls())
print("fitness indvdl 0  = ", best_pop.best_indvdls()[0].fitness)

best_indvdls =  [{'Name1': [[0]], 'Name2': [[0, 0, 0, 0]]}]
fitness indvdl 0  =  0.0


In [28]:
fig = pe.line(x=range(len(best_pop)), y=[ indvdl.fitness for indvdl in best_pop])
fig.show()